In [3]:
# importing necessary libraries for data loading and EDA
import numpy as np, pandas as pd, matplotlib.pyplot as plt, seaborn as sns
import warnings
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from lightgbm import LGBMClassifier
from sklearn.metrics import accuracy_score, auc, precision_score, recall_score, roc_auc_score, roc_curve
from sklearn.metrics import confusion_matrix


warnings.filterwarnings('ignore')

In [4]:
# Loading the train data
df_train = pd.read_csv(r'/home/ducanh/Credit Card Transactions Fraud Detection/Datasets/fraudTrain.csv')
df_test = pd.read_csv(r'/home/ducanh/Credit Card Transactions Fraud Detection/Datasets/fraudTest.csv')
df = pd.concat([df_train, df_test])

In [5]:
df['trans_date_trans_time'] = pd.to_datetime(df['trans_date_trans_time'])
df['trans_hour'] = df['trans_date_trans_time'].dt.time.apply(lambda x: str(x)[:2])

df['dob'] = pd.to_datetime(df['dob'])
df['cust_age'] = df['dob'].dt.year.apply(lambda x: 2021-x)
df['cust_age_groups'] = df['cust_age'].apply(lambda x: 'below 10' if x<10 else ('10-20' if x>=10 and x<20 else ('20-30' if x>=20 and x<30 else('30-40' if x>=30 and x<40 else('40-50' if x>=40 and x<50 else('50-60' if x>=50 and x<60 else('60-70' if x>=60 and x<70 else ('70-80' if x>=70 and x<80 else 'Above 80'))))))))

In [6]:
drop_col = ['Unnamed: 0', 'trans_date_trans_time', 'cc_num', 'merchant','first', 'last', 'street', 'city', 'state', 'lat',
       'long','dob', 'unix_time', 'cust_age', 'merch_lat',
       'merch_long', 'city_pop']
df.drop(drop_col, axis = 1, inplace = True)

In [7]:
age_piv_2 = pd.pivot_table(data = df,
                           index = 'cust_age_groups',
                           columns = 'is_fraud',
                           values = 'amt',
                           aggfunc = np.mean)
age_piv_2.sort_values(by = 1, ascending = True, inplace = True)
age_dic = {k:v for (k,v) in zip(age_piv_2.index.values, age_piv_2.reset_index().index.values)}
df['cust_age_groups'] = df['cust_age_groups'].map(age_dic)

In [8]:
merch_cat = df[df['is_fraud'] == 1].groupby('category')['amt'].mean().sort_values(ascending = True)
merch_cat_dic = {k:v for (k,v) in zip(merch_cat.index.values,merch_cat.reset_index().index.values)}
df['category'] = df['category'].map(merch_cat_dic)

In [9]:
job_txn_piv_2 = pd.pivot_table(data = df,
                               index = 'job',
                               columns = 'is_fraud',
                               values= 'amt',
                               aggfunc = np.mean)
job_cat_dic = {k:v for (k,v) in zip(job_txn_piv_2.index.values, job_txn_piv_2.reset_index().index.values)}
df['job'] = df['job'].map(job_cat_dic)

In [10]:
df['trans_hour'] = df['trans_hour'].astype('int')
df = pd.get_dummies(data  = df, columns = ['gender'], drop_first = True, dtype = 'int')

# Train_test_split

In [11]:
train,test = train_test_split(df, test_size=0.33, random_state=42, stratify = df['is_fraud'])
# visualizing class imbalance
df['is_fraud'].value_counts()
# check
print(train.shape)
print(test.shape)

# let's drop transaction number columns from both the training and testing data
train.drop('trans_num',axis = 1, inplace = True)
test.drop('trans_num',axis = 1, inplace = True)
# splitting data into dependent and independent features respectively
y_train = train['is_fraud']
X_train = train.drop('is_fraud',axis = 1)

y_test = test['is_fraud']
X_test = test.drop('is_fraud',axis = 1)

print('Shape of training data: ',(X_train.shape,y_train.shape))
print('Shape of testing data: ',(X_test.shape,y_test.shape))

(1241103, 9)
(611291, 9)
Shape of training data:  ((1241103, 7), (1241103,))
Shape of testing data:  ((611291, 7), (611291,))


In [12]:
sc = StandardScaler()
# scaling the training and testing data
X_train_sc = sc.fit_transform(X_train)
X_test_sc = sc.transform(X_test)
# convert them into dataframes
X_train_sc = pd.DataFrame(data = X_train_sc, columns = X_train.columns)
X_test_sc = pd.DataFrame(data = X_test_sc, columns = X_test.columns)

# LightGBM

In [14]:
lgbm_model = LGBMClassifier(boosting_type='gbdt', scale_pos_weight=(y_train.value_counts()[0] / y_train.value_counts()[1]), random_state=42, n_estimators=100)
lgbm_model.fit(X_train_sc, y_train)

y_pred_train_proba = lgbm_model.predict_proba(X_train_sc)
y_pred_test_proba = lgbm_model.predict_proba(X_test_sc)

y_train_results = pd.DataFrame(y_pred_train_proba, columns=['pred_not_fraud', 'pred_fraud'])
y_test_results = pd.DataFrame(y_pred_test_proba, columns=['pred_not_fraud', 'pred_fraud'])

y_train_results['y_train_actual'] = y_train.values
y_test_results['y_test_actual'] = y_test.values

[LightGBM] [Info] Number of positive: 6466, number of negative: 1234637
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.005400 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 812
[LightGBM] [Info] Number of data points in the train set: 1241103, number of used features: 7
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.005210 -> initscore=-5.251975
[LightGBM] [Info] Start training from score -5.251975


In [15]:
numbers = [0.1, 0.15, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
for i in numbers:
    y_train_results[i] = y_train_results.pred_fraud.map(lambda x: 1 if x > i else 0)
    y_test_results[i] = y_test_results.pred_fraud.map(lambda x: 1 if x > i else 0)

cutoff_df = pd.DataFrame(columns=['Threshold', 'precision_score', 'recall_score', 'F1_score', 'Accuracy'])

In [16]:
for i in numbers:
    cm1 = confusion_matrix(y_train_results['y_train_actual'], y_train_results[i])
    TP, FP, FN, TN = cm1[1,1], cm1[0,1], cm1[1,0], cm1[0,0]
    
    precision = TP / (TP + FP) if (TP + FP) > 0 else 0
    recall = TP / (TP + FN) if (TP + FN) > 0 else 0
    f1_score_value = (2 * precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    accuracy = (TP + TN) / (TP + TN + FP + FN)
    
    cutoff_df.loc[i] = [i, precision, recall, f1_score_value, accuracy]

print(cutoff_df)

best_idx = cutoff_df['F1_score'].idxmax()
best_threshold = cutoff_df.loc[best_idx, 'Threshold']
best_f1_score = cutoff_df.loc[best_idx, 'F1_score']
best_precision = cutoff_df.loc[best_idx, 'precision_score']
best_recall = cutoff_df.loc[best_idx, 'recall_score']
best_accuracy = cutoff_df.loc[best_idx, 'Accuracy']
best_auc = roc_auc_score(y_train_results['y_train_actual'], y_train_results[best_threshold])

print(f'Best Threshold: {best_threshold:.4f}')
print(f'Best Accuracy: {best_accuracy:.4f}')
print(f'Best Precision: {best_precision:.4f}')
print(f'Best Recall: {best_recall:.4f}')
print(f'Best F1 Score: {best_f1_score:.4f}')
print(f'Best ROC_AUC Score: {best_auc:.4f}')

      Threshold  precision_score  recall_score  F1_score  Accuracy
0.10       0.10         0.077040      0.970770  0.142752  0.939256
0.15       0.15         0.083131      0.969533  0.153132  0.944131
0.20       0.20         0.083812      0.969533  0.154287  0.944625
0.30       0.30         0.091550      0.968141  0.167281  0.949783
0.40       0.40         0.105870      0.967677  0.190858  0.957253
0.50       0.50         0.108805      0.967213  0.195606  0.958555
0.60       0.60         0.114458      0.965512  0.204655  0.960903
0.70       0.70         0.149642      0.953139  0.258672  0.971537
0.80       0.80         0.175303      0.946953  0.295840  0.976514
0.90       0.90         0.183066      0.942314  0.306574  0.977792
Best Threshold: 0.9000
Best Accuracy: 0.9778
Best Precision: 0.1831
Best Recall: 0.9423
Best F1 Score: 0.3066
Best ROC_AUC Score: 0.9601


In [17]:
for i in numbers:
    cm1 = confusion_matrix(y_test_results['y_test_actual'], y_test_results[i])
    TP, FP, FN, TN = cm1[1,1], cm1[0,1], cm1[1,0], cm1[0,0]
    
    precision = TP / (TP + FP) if (TP + FP) > 0 else 0
    recall = TP / (TP + FN) if (TP + FN) > 0 else 0
    f1_score_value = (2 * precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    accuracy = (TP + TN) / (TP + TN + FP + FN)
    
    cutoff_df.loc[i] = [i, precision, recall, f1_score_value, accuracy]

print(cutoff_df)

best_idx = cutoff_df['F1_score'].idxmax()
best_threshold = cutoff_df.loc[best_idx, 'Threshold']
best_f1_score = cutoff_df.loc[best_idx, 'F1_score']
best_precision = cutoff_df.loc[best_idx, 'precision_score']
best_recall = cutoff_df.loc[best_idx, 'recall_score']
best_accuracy = cutoff_df.loc[best_idx, 'Accuracy']
best_auc = roc_auc_score(y_test_results['y_test_actual'], y_test_results[best_threshold])

print(f'Best Threshold: {best_threshold:.4f}')
print(f'Best Accuracy: {best_accuracy:.4f}')
print(f'Best Precision: {best_precision:.4f}')
print(f'Best Recall: {best_recall:.4f}')
print(f'Best F1 Score: {best_f1_score:.4f}')
print(f'Best ROC_AUC Score: {best_auc:.4f}')

      Threshold  precision_score  recall_score  F1_score  Accuracy
0.10       0.10         0.076420      0.958242  0.141552  0.939443
0.15       0.15         0.082597      0.956986  0.152069  0.944395
0.20       0.20         0.083265      0.956986  0.153200  0.944879
0.30       0.30         0.091026      0.955102  0.166211  0.950073
0.40       0.40         0.105278      0.954474  0.189639  0.957498
0.50       0.50         0.108288      0.954160  0.194502  0.958823
0.60       0.60         0.113736      0.952276  0.203202  0.961089
0.70       0.70         0.147909      0.940345  0.255612  0.971464
0.80       0.80         0.172080      0.936578  0.290741  0.976191
0.90       0.90         0.179221      0.931554  0.300608  0.977415
Best Threshold: 0.9000
Best Accuracy: 0.9774
Best Precision: 0.1792
Best Recall: 0.9316
Best F1 Score: 0.3006
Best ROC_AUC Score: 0.9546
